# 创建资产

In [1]:

from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.1.108:9984'  # http
bdb = BigchainDB(bdb_root_url)
print(bdb)

bicycle = {
    'data': {
        'bicycle': {
            'serial_number': 'abcd1234',
            'manufacturer': '1号自行车',
        },
    },
}

metadata = {'planet': 'earth'}

from bigchaindb_driver.crypto import generate_keypair
alice, bob = generate_keypair(), generate_keypair()

prepared_creation_tx = bdb.transactions.prepare(
    operation='CREATE',
    signers=alice.public_key,
    asset=bicycle,
    metadata=metadata,
)

# print(prepared_creation_tx)

fulfilled_creation_tx = bdb.transactions.fulfill(prepared_creation_tx, private_keys=alice.private_key)
print(fulfilled_creation_tx)

# send to bigchaindb node
sent_creation_tx = bdb.transactions.send_commit(fulfilled_creation_tx)
print(sent_creation_tx == fulfilled_creation_tx)


AttributeError: 'str' object has no attribute 'decode'

# 交易

In [27]:
# 交易id
txid = fulfilled_creation_tx['id']
print(txid)

# 检查交易是否成功发送（是否在区块（block）中）,返回事务区块高度
block_height = bdb.blocks.get(txid=txid)
print(block_height)
# 如果事务不在任何区块中，返回None，也可能有其他原因如事务无效，仍在队列中

# 通过快高度检索整个块
block = bdb.blocks.retrieve(str(block_height))
# print(block)

# 资产转移
# 检索交易，通过txid,等价fulfilled_creation_tx
creation_tx = bdb.transactions.retrieve(txid)

issame2 = (creation_tx == fulfilled_creation_tx)
print(issame2)

# 待转移的资产id
assert_id = creation_tx['id']
transfer_asset = {
    'id': assert_id,
}

# 准备转移交易
output_index = 0
output = creation_tx['outputs'][output_index]
transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
        'output_index': output_index,
        'transaction_id': creation_tx['id'],
    },
    'owners_before': output['public_keys'],
}

prepared_transfer_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset=transfer_asset,
    inputs=transfer_input,
    recipients=bob.public_key,
)

# 实现资产转移
fulfilled_transfer_tx = bdb.transactions.fulfill(
    prepared_transfer_tx,
    private_keys=alice.private_key,
)

# 讲交易发送至db节点
sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)

isSame3 = (sent_transfer_tx == fulfilled_transfer_tx)
print(isSame3)

7a09efe754476dc7934011710b63400331aa9fbd59c2617333c80edda0555e6b
868
True
True
